In [1]:
import math
import csv
from heapq import heapify, heappop, heappush

In [2]:
# Load the value of the distances from the provided file
# It is the distance from a star to another
def load_distances(filename):
    # Instead of adjacency list we used a dictionary to store a star,
    # its destination star and the distance between these two stars
    distances = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            source = row[0]
            dest = row[1]
            distance = int(row[2])
            if source not in distances:
                distances[source] = {}
            distances[source][dest] = distance
    return distances

print(load_distances('distances.csv'))

{'Sun': {'Proxima Centauri': 643, 'Lalande 21185': 1059, 'Lacaille 9352': 1194, "Luyten's Star": 1251, 'YZ Ceti': 1766, 'Tau Ceti': 3042, 'Gliese 1061': 2773, 'Wolf 1061': 2501, 'Gliese 876': 2710, '82 G. Eridani': 3247, 'Gliese 581': 2306}, 'Proxima Centauri': {'Sun': 711, 'Lalande 21185': 558, 'Lacaille 9352': 1357, "Luyten's Star": 1351, 'YZ Ceti': 2273, 'Tau Ceti': 2719, 'Gliese 1061': 2940, 'Wolf 1061': 2890, 'Gliese 876': 2490, '82 G. Eridani': 2730, 'Gliese 581': 2751}, 'Lalande 21185': {'Sun': 988, 'Proxima Centauri': 709, 'Lacaille 9352': 1595, "Luyten's Star": 1640, 'YZ Ceti': 2472, 'Tau Ceti': 2902, 'Gliese 1061': 3138, 'Wolf 1061': 3127, 'Gliese 876': 2070, '82 G. Eridani': 2368, 'Gliese 581': 2858}, 'Lacaille 9352': {'Sun': 1203, 'Proxima Centauri': 1357, 'Lalande 21185': 1657, "Luyten's Star": 337, 'YZ Ceti': 2511, 'Tau Ceti': 2095, 'Gliese 1061': 1967, 'Wolf 1061': 2751, 'Gliese 876': 3460, '82 G. Eridani': 3489, 'Gliese 581': 2984}, "Luyten's Star": {'Sun': 1312, 'Proxi

In [3]:
# Load the value of the coordinates from the provided file
# Coordinate of a star in a 3D plane
def load_coordinates(filename):
    coordinates = {}
    with open(filename, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            star = row['Star']
            x = float(row['x'])
            y = float(row['y'])
            z = float(row['z'])
            # (x,y,z) value for one star
            coordinates[star] = (x, y, z)
    return coordinates

print(load_coordinates('Coordinates.csv'))

{'Sun': (0.0, 0.0, 0.0), 'Proxima Centauri': (176.0, -406.0, -49.0), 'Lalande 21185': (552.0, -473.0, -38.0), 'Lacaille 9352': (-848.0, -540.0, 0.0), "Luyten's Star": (-849.0, -542.0, 0.0), 'YZ Ceti': (-280.0, 1568.0, 40.0), 'Tau Ceti': (-1389.0, -2353.0, 182.0), 'Gliese 1061': (-2240.0, -942.0, 838.0), 'Wolf 1061': (-863.0, 1956.0, -409.0), 'Gliese 876': (2378.0, -34.0, -288.0), '82 G. Eridani': (1965.0, -2162.0, -12.0), 'Gliese 581': (-86.0, 2020.0, -48.0), 'Gliese 667 C': (1874.0, 4359.0, -33.0), 'HD 219134': (-1371.0, -5607.0, -52.0), '61 Virginis': (-1879.0, -5265.0, -239.0), 'Gliese 433': (5126.0, -1020.0, -159.0), 'Gliese 357': (-1342.0, -5144.0, 51.0), 'TRAPPIST-1': (2583.0, -9736.0, 67.0), '55 Cancri': (16781.0, 10603.0, -178.0), 'HD 69830': (29087.0, -6996.0, -523.0), 'Upsilon Andromedae': (12339.0, -31372.0, -164.0)}


In [4]:
def euclidean_distance(a, b):
    dx = a[0] - b[0]
    dy = a[1] - b[1]
    dz = a[2] - b[2]

    return math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

In [5]:
def dijkstra(start, end, distances):
    queue = [(0, start)]
    visited = set()
    parents = {}
    distances_from_start = {start: 0}

    while queue:
        current_distance, current_star = heappop(queue)

        if current_star == end:
            break

        if current_star == visited:
            continue

        # After visiting a star it will add it to the visited set.
        # So that, we can keep a track of the visited and unvisited stars
        # It also doesn't allow duplicate as it is a set data structure
        visited.add(current_star)

        # As soon as we start travelling the nodes we add the costs as distance
        for neighbor, distance in distances[current_star].items():
            new_distance = current_distance + distance

            if neighbor not in distances_from_start or new_distance < distances_from_start[neighbor]:
                distances_from_start[neighbor] = new_distance
                parents[neighbor] = current_star
                heappush(queue, (new_distance, neighbor))

    # Checks if there is no known parents for the star we are trying to find the shortest path.
    # Which means tracking the valid path from the start node to the goal node.
    # If true, it means there are no valid paths from the start node to the goal node. Or there maybe
    # are but those are infinite.
    if end not in parents:
        return None, float('inf')

    # Initialize the path with list with the current star
    path = [end]
    # The following code gives us the shortest path by traversing the
    # parents dictionary from end node to the start node.
    # Then reverses the result to show the path from the start to end
    while path[-1] != start:
        path.append(parents[path[-1]])
    path.reverse()

    return path, distances_from_start[end]

In [6]:
distances = load_distances('distances.csv')

dijkstra_path_ua, dijkstra_cost_ua = dijkstra('Sun', '61 Virginis', distances)

print(f"{dijkstra_path_ua}\n\n{dijkstra_cost_ua}")


['Sun', 'Tau Ceti', '61 Virginis']

6362


In [7]:
def astar_search(start, end, coordinates, distances):
    queue = [(0, start)]
    visited = set()
    parents = {}
    distances_from_start = {start: 0}

    while queue:
        current_distance, current_star = heappop(queue)

        if current_star == end:
            break

        if current_star == visited:
            continue

        # After visiting a star it will add it to the visited set.
        # So that, we can keep a track of the visited and unvisited stars
        # It also doesn't allow duplicate as it is a set data structure
        visited.add(current_star)

        # The main algorithm part:
        # It explores the graph, updates the distances and parents for each star,
        # and adds the stars to the queue based on their scores in order to find the
        # lowest cost path. It continues untill the goal node is found
        for neighbor, distance in distances[current_star].items():
            new_distance = current_distance + distance
            # h_score is the heuristic value which is calculated by reading the
            # coordinates file and using the euclidean distance function
            h_score = euclidean_distance(coordinates[neighbor], coordinates[end])
            # A start algorithm: f(n) = g(n) + h(n)
            f_score = new_distance + h_score

            if neighbor not in distances_from_start or new_distance < distances_from_start[neighbor]:
                distances_from_start[neighbor] = new_distance
                parents[neighbor] = current_star
                heappush(queue, (f_score, neighbor))


    # Checks if there is no known parents for the star we are trying to find the shortest path.
    # Which means tracking the valid path from the start node to the goal node.
    # If true, it means there are no valid paths from the start node to the goal node. Or there maybe
    # are but those are infinite.
    if end not in parents:
        return None, float('inf')

    # Initialize the path with list with the current star
    path = [end]
    # The following code gives us the shortest path by traversing the
    # parents dictionary from end node to the start node.
    # Then reverses the result to show the path from the start to end
    while path[-1] != start:
        path.append(parents[path[-1]])
    path.reverse()

    return path, distances_from_start[end]

In [8]:
distances = load_distances('distances.csv')
coordinates = load_coordinates('Coordinates.csv')

astar_path_ua, astar_cost_ua = astar_search('Sun', 'Upsilon Andromedae', coordinates, distances)

print(f"A* Search:\nPath: {astar_path_ua}\nCost: {astar_cost_ua}\n\n")

dijkstra_path_ua, dijkstra_cost_ua = dijkstra('Sun', 'Upsilon Andromedae', distances)

print(f"Dijkstra:\nPath: {dijkstra_path_ua}\nCost: {dijkstra_cost_ua}\n\n")



astar_path_v, astar_cost_v = astar_search('Sun', '61 Virginis', coordinates, distances)

print(f"A* Search:\nPath: {astar_path_v}\nCost: {astar_cost_v}\n\n")

dijkstra_path_v, dijkstra_cost_v = dijkstra('Sun', '61 Virginis', distances)

print(f"Dijkstra:\nPath: {dijkstra_path_v}\nCost: {dijkstra_cost_v}")


A* Search:
Path: None
Cost: inf


Dijkstra:
Path: None
Cost: inf


A* Search:
Path: ['Sun', 'Tau Ceti', '61 Virginis']
Cost: 9344.798182914828


Dijkstra:
Path: ['Sun', 'Tau Ceti', '61 Virginis']
Cost: 6362
